In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    import nsepy as np
except ImportError as err:
    !pip install nsepy
    import nsepy as np
    
try:
    import yfinance as yf
except ImportError as err:
    !pip install yfinance
    import yfinance as yf
    
try:
    import prophet as p
except ImportError as err:
    !pip install prophet
    import prophet as p
    
try:
    import plotly.express as px
    import plotly.graph_objects as go
except ImportError as err:
    !pip install plotly
    import plotly.express as px
    import plotly.graph_objects as go
    
import datetime as DT
import numpy as npy
import pandas as pd
import itertools
import gc

from datetime import date
from nsepy import get_history
from nsepy.derivatives import get_expiry_date
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, DAILY
from datetime import datetime
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [ ]:
# sample options data
nifty_opt = get_history(symbol="NIFTY",
                        start = date(2022,12,1),
                        end=date(2022,12,31),
                        index=True,
                        option_type = 'CE',
                        strike_price=19000,
                        expiry_date = date(2022,12,29)
                       )
nifty_opt.head(10)

**Aim for code**
* Have daily level bull bear grip counter column
* Have aggregated level bull bear counter
* Print trend for the strike price interested in

In [ ]:
counter = 1
expiry_date = ''
while counter > 0:
    expiry_date = input('Enter expiry date in yyyy-mm-dd: ') 
    print("Expirty Date:" + expiry_date)
    user_ip = input("Press 1 if information is correct:")
    if int(user_ip)==1:
        counter = 0
expiry = DT.datetime.strptime(expiry_date, "%Y-%m-%d").date()
print(expiry)
type(expiry)

In [ ]:
from datetime import date
end_date = date.today()
print(end_date.strftime("%Y-%m-%d"))
start_date = end_date
flag = True
while flag:
    start_date = start_date - DT.timedelta(days=1)
    print(start_date.strftime("%Y-%m-%d"))
    if(npy.busday_count(start_date, end_date)>=7):
        flag = False
        break;
        
df_nifty = yf.download("^NSEI", start=start_date, end=end_date)
close_price = float(df_nifty.tail(1)['Close'])
print(type(end_date))
print(type(start_date))

In [ ]:
close_price_range = close_price*0.03
close_price_lower_limit = round(close_price - close_price_range,-2)
close_price_upper_limit = round(close_price + close_price_range,-2)
print(close_price_lower_limit)
print(close_price_upper_limit)

In [ ]:
df_call_values = pd.DataFrame()
df_put_values = pd.DataFrame()

In [ ]:
def option_values_sp(symbol,start_date,end_date,call_put,strike,expiry,df_opt):
    nifty_opt = get_history(symbol=symbol,
                        start = start_date,
                        end = end_date,
                        index=True,
                        option_type = call_put,
                        strike_price=strike,
                        expiry_date = expiry)
    nifty_opt['DateVal'] = nifty_opt.index
    print("For strike price: " + call_put + " " + str(strike) + " shape is " + str(nifty_opt.shape[0]))
    if df_opt.shape[0]<=0:
        df_opt = nifty_opt
        print("Empty Dataframe")
    else:
        df_opt = df_opt.append(nifty_opt, ignore_index=True)
        
    return df_opt

In [ ]:
for sp in range(int(close_price_lower_limit),int(close_price_upper_limit)+50,50):
    print(sp)
    df_call_values = option_values_sp('NIFTY',start_date,end_date,'CE',sp,expiry,df_call_values)
    df_put_values = option_values_sp('NIFTY',start_date,end_date,'PE',sp,expiry,df_call_values)

In [ ]:
print(df_call_values.shape)
print(df_put_values.shape)

In [ ]:
df_put_values['DateVal'] = df_put_values['DateVal'].astype(str)
df_put_values['KEY'] = df_put_values['DateVal'] + "_" + df_put_values['Strike Price'].astype(int).astype(str)

In [ ]:
df_put_values.head()

In [ ]:
df_call_values['DateVal'] = df_call_values['DateVal'].astype(str)
df_call_values['KEY'] = df_call_values['DateVal'] + "_" + df_call_values['Strike Price'].astype(int).astype(str)

In [ ]:
df_call_values.head()

In [ ]:
df_overall = df_call_values
df_overall = df_overall.merge(df_put_values,"inner",'KEY')

In [ ]:
df_overall['Date'] = df_overall['KEY'].str[:10]

In [ ]:
lst_dates = df_overall['Date'].unique()

In [ ]:
lst_unique_dates = df_overall.index.unique()

In [ ]:
df_overall.head()

In [ ]:
df_overall.shape

In [ ]:
df_overall.dtypes

In [ ]:
score_bull = 0
score_bear = 0
lst_score_bull = []
lst_score_bear = []
lst_pcr = []
lst_delta_oi = []
lst_date = []
lst_turnover_delta = []
lst_preium_delta = []

In [ ]:
for dateval in lst_dates:
    put_oi = 0.0
    call_oi = 0.0
    pcr = 0.0
    
    df_option_chain = df_overall.loc[df_overall['Date']==dateval]
    print(df_option_chain.shape)
    call_oi = df_option_chain['Open Interest_x'].sum()
    put_oi = df_option_chain['Open Interest_y'].sum()
    delta_call_oi = df_option_chain['Change in OI_x'].sum()
    delta_put_oi = df_option_chain['Change in OI_y'].sum()
    call_turnover = df_option_chain['Turnover_x'].sum()
    put_turnover = df_option_chain['Turnover_y'].sum()
    call_premium = df_option_chain['Premium Turnover_x'].sum()
    put_premium = df_option_chain['Premium Turnover_y'].sum()
    
    pcr = round(put_oi / call_oi, 4)
    delta_oi_buildup = round(delta_put_oi / delta_call_oi,4)
    turnover_pcr = round(put_turnover / call_turnover,4)
    premium_pcr = round(put_premium / call_premium, 4)
    
    lst_pcr.append(pcr)
    lst_delta_oi.append(delta_oi_buildup)
    lst_date.append(dateval)
    lst_turnover_delta.append(turnover_pcr)
    lst_preium_delta.append(premium_pcr)

In [ ]:
df_analysis = pd.DataFrame()
df_analysis['PCR'] = lst_pcr
df_analysis['OI_CHANGE_PCR'] = lst_delta_oi
df_analysis['DATE'] = lst_date
df_analysis['TURNOVER_PCR'] = lst_turnover_delta
df_analysis['PREMIUM_PCR'] = lst_preium_delta

df_analysis.head(10)